# **Laboratorio 12: 🚀 Despliegue 🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Francisca Ruiz
- Nombre de alumno 2: Valentina Zúñiga

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/vazuniga1/Laboratorios)

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow (2.0 puntos)**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `f1-score` se registre como `"valid_f1"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

In [ ]:
!pip install mlflow
!pip install -U kaleido
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 459.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 9.8 MB/s eta 0:00:00


In [ ]:
import os
import pickle
import optuna
import mlflow
import pandas as pd
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
!wget 'https://docs.google.com/uc?export=download&id=1Ry2kgZGMKn76S976uHw4vTwvhnZlBGzq' -O water_potability.csv

--2024-11-26 00:08:41--  https://docs.google.com/uc?export=download&id=1Ry2kgZGMKn76S976uHw4vTwvhnZlBGzq
Resolving docs.google.com (docs.google.com)... 173.194.202.113, 173.194.202.100, 173.194.202.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.202.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Ry2kgZGMKn76S976uHw4vTwvhnZlBGzq&export=download [following]
--2024-11-26 00:08:41--  https://drive.usercontent.google.com/download?id=1Ry2kgZGMKn76S976uHw4vTwvhnZlBGzq&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.199.132, 2607:f8b0:400e:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.199.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 525187 (513K) [application/octet-stream]
Saving to: ‘water_potability.csv’

water_potability.cs 100%[===================>] 512.88K  --.-

In [ ]:
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model

In [ ]:
# Se cargan los datos
df = pd.read_csv('water_potability.csv')
X, y = df.drop("Potability", axis=1), df["Potability"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configurar el directorio para los artefactos
artifacts_dir = "mlruns/artifacts"
os.makedirs(artifacts_dir, exist_ok=True)

def optimize_model(trial):
      params = {
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "eta": trial.suggest_float('eta', 0.01, 0.1),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            }

      # Se crea  el modelo
      model = xgb.XGBClassifier(**params)

      name_run = f"XGBoost con lr {params['eta']}"
      # Registrar los resultados en MLflow
      with mlflow.start_run(run_name = name_run):
        # se entrena el modelo
        model.fit(X_train, y_train)

        # Se predice y calcula el f1
        y_pred = model.predict(X_test)
        valid_f1 = f1_score(y_test, y_pred, average="weighted")

        mlflow.log_params(params)
        mlflow.log_metric("valid_f1", valid_f1)

      return valid_f1

def main():
    mlflow.set_experiment("XGBoost Potability Experiment")

    # Se crea un estudio de Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(optimize_model, n_trials=300)

    best_trial = study.best_trial

    # Guardar el mejor modelo
    models_dir = "models"
    os.makedirs(models_dir, exist_ok=True)

    best_params = best_trial.params
    best_model = xgb.XGBClassifier(**best_params)
    best_model.fit(X_train, y_train)

    model_path = "models/best_model.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(best_model, f)
    mlflow.log_artifact('models/best_model.pkl')

    # Guardar gráficos de Optuna
    plots_dir = "plots"
    os.makedirs(plots_dir, exist_ok=True)

    optuna.visualization.matplotlib.plot_optimization_history(study)
    plt.savefig(f"{plots_dir}/optimization_history.png")
    plt.close()

    optuna.visualization.matplotlib.plot_param_importances(study)
    plt.savefig(f"{plots_dir}/param_importance.png")
    plt.close()

    # Loggear gráficos en MLflow
    mlflow.log_artifact(f"{plots_dir}/optimization_history.png")
    mlflow.log_artifact(f"{plots_dir}/param_importance.png")

    # Guardar la importancia de características
    xgb.plot_importance(best_model, importance_type="weight")
    plt.savefig(f"{artifacts_dir}/feature_importance.png")
    plt.close()

    # Registrar las versiones de las librerías
    mlflow.log_artifacts(artifacts_dir)
    mlflow.set_tag("mlflow.version", mlflow.__version__)
    mlflow.set_tag("optuna.version", optuna.__version__)
    mlflow.set_tag("xgboost.version", xgb.__version__)

    return best_model


if __name__ == "__main__":
    best_model = main()

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 328, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 422, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 1368, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 1361, in _read_helper
    result = read_yaml(root, file_name)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingConfigException: Yaml file '/content/mlruns/artifacts/meta.yaml' doe

In [ ]:
mlflow.end_run()

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

**`HINT:` Recuerde que puede utilizar [http://localhost:8000/docs](http://localhost:8000/docs) para hacer un `POST`.**

In [ ]:
!pip install "fastapi[all]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 11.4 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel

class WaterPotability(BaseModel):
    ph: float
    Hardness: float
    Solids: float
    Chloramines: float
    Sulfate: float
    Conductivity: float
    Organic_carbon: float
    Trihalomethanes: float
    Turbidity: float

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
from utils import WaterPotability
import pickle
import os


# inicializamos API
app = FastAPI()

# Se define ruta del mejor modelo y se carga
PATH_BEST_MODEL = "models/best_model.pkl"
with open(PATH_BEST_MODEL, 'rb') as f:
    best_model = pickle.load(f)

# Defina GET con ruta tipo home que describa brevemente su modelo, \
# el problema que intenta resolver, su entrada y salida.
@app.get('/', response_class=HTMLResponse) # ruta
async def home():
    html_response = """
   <html>
    <h1>Modelo de Machine Learning para predecir la calidad del agua</h1>
    <p>Este modelo de Machine Learning se encarga de predecir la calidad del agua en base a diferentes features.</p>

    <body>
        <p> Ejemplo de entrada del modelo: </p>

    <p style="text-align: center;">
            "ph":10.316400384553162, <br>
            "Hardness":217.2668424334475, <br>
            "Solids":10676.508475429378, <br>
            "Chloramines":3.445514571005745, <br>
            "Sulfate":397.7549459751925, <br>
            "Conductivity":492.20647361771086, <br>
            "Organic_carbon":12.812732207582542, <br>
            "Trihalomethanes":72.28192021570328, <br>
            "Turbidity":3.4073494284238364 <br>
    </p>

    <p> Ejemplo de salida del modelo: </p>
    <p style="text-align: center;"> "potabilidad": 0 </p>
     </body>
    </html>
"""
    return html_response


@app.post('/predict') # ruta
async def predict(data: WaterPotability):
    features = [data.ph, data.Hardness, data.Solids, data.Chloramines, data.Sulfate, \
        data.Conductivity, data.Organic_carbon, data.Trihalomethanes, data.Turbidity]

    # se realiza la predicción
    prediction = best_model.predict([features])

    html_content = f"""
    <html>
    <h1>Resultado de la predicción</h1>
    <p>El modelo predice que la calidad del agua es: {prediction[0]}</p>
    </html>
    """

    return HTMLResponse(content=html_content, status_code=200)

# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.


- Respuestas:

4. Se agregan las imágenes de la ejecución del contenedor junto con predicciones para el ejemplo (y su resultado) (docker.jpg) y los recursos que utiliza (usage.png)

5. Al ver el uso que se le da, se puede notar que este tipo de contenedor es bastante liviano ya que no utiliza tantos recursos (como CPU, RAM) debido a la naturaleza de este, ya que simplemente es una API con un modelo liviano.


## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- ¿Cómo se diferencia Docker de una máquina virtual (VM)?

Si bien ambas son tecnologías de virtualización, se diferencian en la forma en que gestionan los recursos. Las VM ejecutan un sistema operativo completo (como Linux, Windows, MacOS, etc) en un hipervisor. Por otro lado, Docker utiliza contenedores que permite que las aplicaciones se ejecuten de manera aislada pero compartiendo el mismo kernel del sistema operativo anfitrión. Esto hace que Docker sea más ligero, más rápido de iniciar y menos demandante en recursos.


- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?

Cuando se ejecuta una aplicación directamente en el sistema local, hay una dependencia de las configuraciones espécificas del entorno, lo que podría causar inconsistencias al mover la aplicación a otro sistema. En cambio, Docker encapsula la aplicación con todas sus dependencias en un contenedor, asegurando que se ejctue igual en cualquier máquina con Docker instalado.


- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?

Docker asegura la consistencia entre distintos entornos mediante la creación de imagenes de contenedor. Al ejecutar la misma imagen en desarrollo y producción, se garantiza que el comportamiento sea idéntico, independiente de las diferencias en los sistemas subyacentes.


- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?

Los volúmenes en Docker son utilizados para persistir datos generados y utilizados por contenedores, estos son directorios montados desde el sistema host o son volúmenes administrados por Docker. Con esto, se garantiza que los datos no se pierdan cuando un contenedor se elimina o se recrea.

- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

Un Dockerfile es un archivo de texto que define los pasos para crear una imagen de Docker, especificando el sistema base, las dependencias y configuraciones necesarias para ejecutar la aplicación. Su propósito es automatizar y estandarizar el proceso de construcción de imágenes Docker. Por otro lado, un docker-compose.yml es un archivo de configuración utilizado por Docker Compose para definir y ejecutar aplicaciones multicontenedor. Este archivo describe los servicios, redes y volúmenes necesarios para la aplicación, lo que permite orquestar fácilmente múltiples contenedores y gestionar sus interacciones.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>